# 1. Data loading and preprocessing

In [9]:
import pandas as pd

In [ ]:
import pandas as pd

# Ruta al archivo
directory = r'data_interfaz/clusterAtractividadJuntos_2024.csv'

# Cargar CSV
df = pd.read_csv(directory)

# Rellenar grupo vacío con 1
df["grupo"] = df["grupo"].fillna(1)

# Eliminar duplicados por "Nombre", conservando la primera ocurrencia
df = df.drop_duplicates(subset=["Nombre"], keep="first")

# Lista de municipios a eliminar
municipios_a_eliminar = ["Villanueva de la Calzada", "Las Rozas", "Madrid"]

# Filtrar fuera esos municipios
df = df[~df["Nombre"].isin(municipios_a_eliminar)]

# Mostrar resultado
print(df.head())

# (Opcional) Guardar el archivo limpio si lo necesitas
# df.to_csv(directory, index=False)


              Nombre  contratos_100  ocupadosColectivos_100  paro_total  \
0       Acebeda (La)         100.00                   56.24       77.42   
1            Ajalvir          78.23                  100.00       64.77   
2  Alameda del Valle          49.91                   53.85       74.87   
3         Alamo (El)          44.57                   46.05       41.29   
4  Alcalá de Henares          58.56                   46.33       28.61   

   pensionistas_100  pension_media  bibliotecas_100_alumnos  \
0              3.18           0.00                    49.07   
1             72.26          29.35                    67.83   
2             45.17          46.32                    49.07   
3             45.45          62.07                    46.39   
4             33.27          74.77                    67.54   

   centros_priv_100_alumnos_priv  centros_pub_100_alumnos_pub  \
0                          48.96                        45.03   
1                          43.45        

In [12]:
len(df)

178

In [13]:
df.iloc(0)

In [14]:
blocks = {
    "educacion": [
        'bibliotecas_100_alumnos',
        'centros_priv_100_alumnos_priv',
        'centros_pub_100_alumnos_pub',
        'extraescolares_10_alumnos',
        'idiomas_10_alumnos',
        'profesores_priv_100_alumnos_priv',
        'profesores_pub_100_alumnos_pub',
        'unidades_escolares_priv_100_alumnos_priv',
        'unidades_escolares_publicas_por_100_alumnos',
    ],
    "salud": [
        'centros_100',
        'farmacias_100',
        'centrosSociales_10',
        'clinicaDental_10',
        'consultaPrimaria_100',
        'otroConsulta_100',
        'orgNoSanitaria_total_100',
    ],
    "transporte": [
        'bus_100',
        'estaciontren_100',
        'servicioCoches_100',
        'estacionbus_100',
        'distanciaCentro',
    ],
    "economia": [
        'contratos_100',
        'ocupadosColectivos_100',
        'paro_total',
        'pensionistas_100',
        'pension_media',
    ],
    "housing": [
        'Familiares',
        'superficieFamiliares',
        'Habitaciones',
        'Banos',
        'Precio',
    ]
}

column_types = {
    "educacion": {
        "colsHigher": blocks["educacion"],
        "colsLower": []
    },
    "salud": {
        "colsHigher": blocks["salud"],
        "colsLower": []
    },
    "transporte": {
        "colsHigher": [
            'bus_100',
            'estaciontren_100',
            'servicioCoches_100',
            'estacionbus_100'
        ],
        "colsLower": ['distanciaCentro']
    },
    "economia": {
        "colsHigher": [
            'contratos_100',
            'ocupadosColectivos_100',
            'pension_media'
        ],
        "colsLower": [
            'paro_total',
            'pensionistas_100'
        ]
    },
    "housing": {
        "colsHigher": [
            'Familiares',
            'superficieFamiliares',
            'Habitaciones',
            'Banos'
        ],
        "colsLower": ['Precio']
    }
}


In [15]:
# Iterar sobre los tipos de columnas
for block, columns in column_types.items():
    # Obtener las columnas que necesitan inversa
    cols_lower = columns["colsLower"]
    print("cols_lower of ", block, ":", cols_lower)
    # Aplicar la transformación
    for col in cols_lower:
        if col in df.columns:  # Verificar que la columna exista en el DataFrame
            df[col] = 100 - df[col]

cols_lower of  educacion : []
cols_lower of  salud : []
cols_lower of  transporte : ['distanciaCentro']
cols_lower of  economia : ['paro_total', 'pensionistas_100']
cols_lower of  housing : ['Precio']


# 2. Data normalization

Los intervalos son con los datos del mismo municipio. La normalización de valor / (max-min) es con los valores individuales de todos los municipios.

De tal manera que si tenemos:

| Municipio   | Indicador 1 (Bloque A) | Indicador 2 (Bloque A) | Indicador 3 (Bloque B) | Indicador 4 (Bloque B) |
|-------------|-------------------------|-------------------------|-------------------------|-------------------------|
| Municipio 1 | 78                      | 45                      | 62                      | 89                      |
| Municipio 2 | 80                      | 56                      | 91                      | 73                      |


El municipio 1, bloque A estaría entre el intervalo [45, (45+78)/2] pero para normalizarlo usaríamos como min el 45 y máximo el 80 del municipio 2.

## 2.1. Intervalos por bloque

In [16]:
def calculate_intervals(df, blocks):
    results = []

    for municipio, group in df.groupby('Nombre'):
        # Obtener el grupo asociado al municipio
        grupo_value = group['grupo'].iloc[0] if 'grupo' in group.columns else None

        municipio_result = {'Nombre': municipio, 'grupo': grupo_value}

        for block_name, columns in blocks.items():
            # Filtrar las columnas del bloque que están en el DataFrame, excluyendo "grupo"
            block_columns = [col for col in columns if col in df.columns and col != 'grupo']

            if block_columns:  # Asegurarse de que hay columnas válidas
                block_data = group[block_columns]

                # Calcular el mínimo y la media
                min_value = block_data.min().min()
                mean_value = block_data.mean().mean()

                municipio_result[block_name] = [min_value, mean_value]
            else:
                municipio_result[block_name] = None  # No hay datos para este bloque

        results.append(municipio_result)

    return pd.DataFrame(results)

intervalos_df = calculate_intervals(df, blocks)
intervalos_df.head()

,Nombre,grupo,educacion,salud,transporte,economia,housing
0,Acebeda (La),1.0,"[26.72, 44.68555555555555]","[38.36, 44.21714285714285]","[47.43, 70.25399999999999]","[0.0, 55.128]","[14.3, 35.544]"
1,Ajalvir,2.0,"[33.08, 48.47777777777778]","[26.18, 44.31142857142857]","[40.27, 52.86]","[27.739999999999995, 54.110000000000014]","[18.53, 32.476]"
2,Alameda del Valle,1.0,"[26.72, 44.68555555555555]","[44.3, 53.63285714285714]","[47.4, 55.898]","[25.129999999999995, 46.007999999999996]","[40.26, 61.470000000000006]"
3,Alamo (El),2.0,"[42.5, 53.98222222222223]","[22.69, 34.027142857142856]","[43.56, 46.38199999999999]","[44.57, 53.19]","[32.3, 40.254000000000005]"
4,Alcalá de Henares,3.0,"[29.35, 46.166666666666664]","[42.22, 58.605714285714285]","[19.12, 45.212]","[46.33, 63.556]","[34.18, 52.727999999999994]"


In [17]:
intervalos_df[intervalos_df["Nombre"] == "Quijorna"]

,Nombre,grupo,educacion,salud,transporte,economia,housing
114,Quijorna,2.0,"[34.09, 68.19666666666667]","[42.13, 52.278571428571425]","[40.59, 45.92]","[19.24, 33.230000000000004]","[34.5, 50.064]"


## 2.2. Valor único por bloque

In [18]:
valores_unicos_df = pd.DataFrame(index=df.index)  # Asegurarse de que el índice sea el mismo que df

# Crear un diccionario para almacenar los valores mínimo y máximo por bloque y por grupo
min_max_by_block_and_group = {}

# Iterar sobre los grupos
for group in df['grupo'].unique():
    group_data = df[df['grupo'] == group]  # Filtrar los datos del grupo actual

    # Crear un diccionario para almacenar los resultados por bloque
    min_max_by_block = {}

    print(f"Procesando grupo {group}...")  # Print para verificar qué grupo estamos procesando

    # Calcular la media por bloque para cada municipio
    for block, indicators in blocks.items():
        # Seleccionar solo las columnas relevantes para el bloque que están en el DataFrame
        relevant_columns = [col for col in indicators if col in group_data.columns]

        if not relevant_columns:  # Si no hay columnas relevantes, saltar este bloque
            print(f"  No se encontraron columnas relevantes para el bloque {block} en el grupo {group}.")
            continue

        # Calcular la media para cada fila (municipio)
        mean_values = group_data[relevant_columns].mean(axis=1)

        # Normalizar las medias por el rango máximo-mínimo del bloque
        block_min = group_data[relevant_columns].min().min()  # Mínimo global del bloque
        block_max = group_data[relevant_columns].max().max()  # Máximo global del bloque

        if block_max == block_min:
            print(f"  Advertencia: El rango del bloque {block} para el grupo {group} es cero (max = min).")

        normalized_mean_values = (mean_values) / (block_max - block_min)

        # Asegurarse de que los índices coinciden al asignar los valores normalizados
        valores_unicos_df.loc[group_data.index, block] = normalized_mean_values

        # Almacenar los valores mínimo y máximo para el bloque
        min_max_by_block[block] = {
            "min": block_min,
            "max": block_max
        }

    # Almacenar los resultados para el grupo actual
    min_max_by_block_and_group[group] = min_max_by_block

# Mantener la columna 'grupo' en el DataFrame
valores_unicos_df['grupo'] = df['grupo']

# Añadir la columna 'Nombre' al nuevo DataFrame
valores_unicos_df['Nombre'] = df['Nombre']

# Reordenar las columnas para que 'Nombre' y 'grupo' estén al principio
valores_unicos_df = valores_unicos_df[['Nombre', 'grupo'] + list(valores_unicos_df.columns[:-2])]

# Mostrar los valores de mínimo y máximo por bloque y grupo
print("\nResultados finales:")
for group, blocks in min_max_by_block_and_group.items():
    print(f"Grupo {group}:")
    for block_name, values in blocks.items():
        print(f"  {block_name}:")
        print(f"    Mínimo = {values['min']}")
        print(f"    Máximo = {values['max']}")

# Mostrar el nuevo DataFrame con las medias normalizadas
print("\nDataFrame final con medias normalizadas:")
print(valores_unicos_df)

Procesando grupo 1.0...
Procesando grupo 2.0...
Procesando grupo 3.0...

Resultados finales:
Grupo 1.0:
  educacion:
    Mínimo = 26.72
    Máximo = 100.0
  salud:
    Mínimo = 38.36
    Máximo = 100.0
  transporte:
    Mínimo = 7.099999999999994
    Máximo = 100.0
  economia:
    Mínimo = 0.0
    Máximo = 100.0
  housing:
    Mínimo = 0.0
    Máximo = 100.0
Grupo 2.0:
  educacion:
    Mínimo = 16.9
    Máximo = 100.0
  salud:
    Mínimo = 22.69
    Máximo = 100.0
  transporte:
    Mínimo = 22.290000000000006
    Máximo = 100.0
  economia:
    Mínimo = 0.0
    Máximo = 100.0
  housing:
    Mínimo = 1.23
    Máximo = 100.0
Grupo 3.0:
  educacion:
    Mínimo = 0.0
    Máximo = 100.0
  salud:
    Mínimo = 0.0
    Máximo = 100.0
  transporte:
    Mínimo = 6.04
    Máximo = 100.0
  economia:
    Mínimo = 0.0
    Máximo = 100.0
  housing:
    Mínimo = 19.3
    Máximo = 100.0

DataFrame final con medias normalizadas:
                    Nombre  grupo  educacion     salud  transporte  economia

## 2.3. Intervalos totales

In [19]:
def add_total_interval(valores_unicos_df, intervalos_df):
    # Excluir las columnas "Municipio" y "grupo" antes de calcular el intervalo
    filtered_df = valores_unicos_df.drop(columns=['Nombre', 'grupo'], errors='ignore')

    # Calcular [min, media] por fila
    total_intervals = filtered_df.apply(lambda row: [row.min(), row.mean()], axis=1)

    # Añadir la nueva columna 'total' al DataFrame intervalos_df
    intervalos_df['total'] = total_intervals.values

    return intervalos_df

intervalos_df = add_total_interval(valores_unicos_df, intervalos_df)
intervalos_df.head()

,Nombre,grupo,educacion,salud,transporte,economia,housing,total
0,Acebeda (La),1.0,"[26.72, 44.68555555555555]","[38.36, 44.21714285714285]","[47.43, 70.25399999999999]","[0.0, 55.128]","[14.3, 35.544]","[0.35544, 0.5980178860496119]"
1,Ajalvir,2.0,"[33.08, 48.47777777777778]","[26.18, 44.31142857142857]","[40.27, 52.86]","[27.739999999999995, 54.110000000000014]","[18.53, 32.476]","[0.32880429280145795, 0.5413315875034939]"
2,Alameda del Valle,1.0,"[26.72, 44.68555555555555]","[44.3, 53.63285714285714]","[47.4, 55.898]","[25.129999999999995, 46.007999999999996]","[40.26, 61.470000000000006]","[0.46007999999999993, 0.6312741979695434]"
3,Alamo (El),2.0,"[42.5, 53.98222222222223]","[22.69, 34.027142857142856]","[43.56, 46.38199999999999]","[44.57, 53.19]","[32.3, 40.254000000000005]","[0.4075529006783437, 0.5252115084130067]"
4,Alcalá de Henares,3.0,"[29.35, 46.166666666666664]","[42.22, 58.605714285714285]","[19.12, 45.212]","[46.33, 63.556]","[34.18, 52.727999999999994]","[0.46166666666666667, 0.563570038296994]"


## 2.4. Valores individuales totales

In [20]:
valores_unicos_df

,Nombre,grupo,educacion,salud,transporte,economia,housing
0,Acebeda (La),1.0,0.609792,0.717345,0.756233,0.55128,0.355440
1,Ajalvir,2.0,0.583367,0.573166,0.680221,0.54110,0.328804
2,Alameda del Valle,1.0,0.609792,0.870098,0.601701,0.46008,0.614700
3,Alamo (El),2.0,0.649606,0.440139,0.596860,0.53190,0.407553
4,Alcalá de Henares,3.0,0.461667,0.586057,0.481183,0.63556,0.653383
...,...,...,...,...,...,...,...
173,Villar del Olmo,1.0,0.890996,0.738157,0.422239,0.50248,0.669540
174,Villarejo de Salvanés,2.0,0.588755,0.577600,0.653069,0.64980,0.406783
175,Villaviciosa de Odón,2.0,0.557066,0.732284,0.575138,0.55472,0.825554
176,Villavieja del Lozoya,1.0,0.709683,0.847733,0.598385,0.42454,0.378420


In [21]:
# 1. Calcular la media por fila (excluyendo 'Municipio' y 'grupo')
valores_unicos_df['mean_per_row'] = valores_unicos_df.drop(['Nombre', 'grupo'], axis=1).mean(axis=1)  # media por fila

# 2. Calcular el mínimo y máximo global por grupo (excluyendo 'mean_per_row')
# Seleccionar las columnas restantes sin contar 'Municipio', 'grupo', y 'mean_per_row'
columns_of_interest = valores_unicos_df.drop(columns=['Nombre', 'grupo', 'mean_per_row']).columns

# Calcular el mínimo y máximo de cada columna de interés por grupo
group_min = valores_unicos_df.groupby('grupo')[columns_of_interest].transform('min')
group_max = valores_unicos_df.groupby('grupo')[columns_of_interest].transform('max')

# 3. Normalizar la media por fila usando los valores máximos y mínimos globales por grupo
# Usamos group_min y group_max para cada grupo
# La normalización es: (mean_per_row - min del grupo) / (max del grupo - min del grupo)
valores_unicos_df['total'] = (valores_unicos_df['mean_per_row']) / (group_max.max(axis=1) - group_min.min(axis=1))

# 4. Opcional: Drop la columna auxiliar 'mean_per_row' si ya no la necesitas
# valores_unicos_df = valores_unicos_df.drop(columns=['mean_per_row'], errors='ignore')

# Mostrar los primeros resultados
valores_unicos_df.head()


,Nombre,grupo,educacion,salud,transporte,economia,housing,mean_per_row,total
0,Acebeda (La),1.0,0.609792,0.717345,0.756233,0.55128,0.355440,0.598018,0.729153
1,Ajalvir,2.0,0.583367,0.573166,0.680221,0.54110,0.328804,0.541332,0.678306
2,Alameda del Valle,1.0,0.609792,0.870098,0.601701,0.46008,0.614700,0.631274,0.769702
3,Alamo (El),2.0,0.649606,0.440139,0.596860,0.53190,0.407553,0.525212,0.658107
4,Alcalá de Henares,3.0,0.461667,0.586057,0.481183,0.63556,0.653383,0.563570,0.668561


# Exports

In [22]:
intervalos_df.head()

,Nombre,grupo,educacion,salud,transporte,economia,housing,total
0,Acebeda (La),1.0,"[26.72, 44.68555555555555]","[38.36, 44.21714285714285]","[47.43, 70.25399999999999]","[0.0, 55.128]","[14.3, 35.544]","[0.35544, 0.5980178860496119]"
1,Ajalvir,2.0,"[33.08, 48.47777777777778]","[26.18, 44.31142857142857]","[40.27, 52.86]","[27.739999999999995, 54.110000000000014]","[18.53, 32.476]","[0.32880429280145795, 0.5413315875034939]"
2,Alameda del Valle,1.0,"[26.72, 44.68555555555555]","[44.3, 53.63285714285714]","[47.4, 55.898]","[25.129999999999995, 46.007999999999996]","[40.26, 61.470000000000006]","[0.46007999999999993, 0.6312741979695434]"
3,Alamo (El),2.0,"[42.5, 53.98222222222223]","[22.69, 34.027142857142856]","[43.56, 46.38199999999999]","[44.57, 53.19]","[32.3, 40.254000000000005]","[0.4075529006783437, 0.5252115084130067]"
4,Alcalá de Henares,3.0,"[29.35, 46.166666666666664]","[42.22, 58.605714285714285]","[19.12, 45.212]","[46.33, 63.556]","[34.18, 52.727999999999994]","[0.46166666666666667, 0.563570038296994]"


In [23]:
valores_unicos_df.head()

,Nombre,grupo,educacion,salud,transporte,economia,housing,mean_per_row,total
0,Acebeda (La),1.0,0.609792,0.717345,0.756233,0.55128,0.355440,0.598018,0.729153
1,Ajalvir,2.0,0.583367,0.573166,0.680221,0.54110,0.328804,0.541332,0.678306
2,Alameda del Valle,1.0,0.609792,0.870098,0.601701,0.46008,0.614700,0.631274,0.769702
3,Alamo (El),2.0,0.649606,0.440139,0.596860,0.53190,0.407553,0.525212,0.658107
4,Alcalá de Henares,3.0,0.461667,0.586057,0.481183,0.63556,0.653383,0.563570,0.668561


In [ ]:
from os import mkdir

intervalos_df.to_csv(r"data_interfaz/intervalos_ia.csv")
valores_unicos_df.to_csv(r"data_interfaz/valores.csv")